In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from random import random
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.model_selection import train_test_split
import scipy.io
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

with open("letter-recognition.data") as myfile:
    data = myfile.read()

label = []
data = data.splitlines()
for i in range(len(data)):
    data[i] = data[i].split(",")
    label.append(data[i][:1])
    data[i] = data[i][1:]
data = np.array(data)
data = data.astype(int)

num_letters = np.zeros(26)
for i in range(len(label)):
    label[i] = ord(label[i][0]) - 65
    num_letters[label[i]] += 1
label = np.array(label)
label = to_categorical(label)

In [ ]:
y_pos = np.arange(26)
objects = map(chr, range(65, 91))
plt.bar(y_pos, num_letters, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Broj Slova')
plt.title('Podela podataka')

In [ ]:
data = preprocessing.scale(data)
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size = 0.2)

In [ ]:
struktura = np.array([[10,15] , [20,50,10]])
regularizacija = np.array([0.1 , 0.25, 0.5, 0.7])
tezine = np.array([2, 3, 4])
fje = np.array(['tanh' , 'relu' , 'sigmoid'])
maxacc = 0 
history = 0
arg = []
t = 0

In [ ]:
for stru in struktura:
    for reg in regularizacija:
        for tez in tezine:
            for f in fje:
                model = keras.Sequential()
                
                model.add(keras.layers.Dense(stru[0], input_shape = (16,), activation = f , kernel_regularizer=keras.regularizers.l2(l=reg)))
                for i in range(len(stru)-1):
                    model.add(keras.layers.Dense(stru[i+1], activation = f , kernel_regularizer=keras.regularizers.l2(l=reg)))
                    b = [tez] * stru[i+1]
                    w = [b] * stru[i]
                    w = np.array(w)
                    b = np.zeros(stru[i+1])
                    model.layers[i+1].set_weights([w,b])
                
                earlystopper = EarlyStopping(monitor='val_loss', min_delta=4, patience=10, verbose=1, mode='auto')
                model.add(keras.layers.Dense(26, activation = "softmax"))     
                model.compile(optimizer = "adam", loss='categorical_crossentropy', metrics=['accuracy'])
                hist = model.fit(x_train, y_train, epochs = 300, validation_split = 0.15, batch_size = 800, verbose = 0,callbacks = [earlystopper])
                loss, acc = model.evaluate(x_test, y_test, verbose = 0)
                if(acc > maxacc):
                    maxacc=acc
                    arg = [stru, reg, tez, f, loss, acc]
                    history = hist
                print(t,acc)
                t = t+1


In [ ]:
history_dict=history.history

loss_values = history_dict['loss']
val_loss_values=history_dict['val_loss']
plt.figure()
plt.figure()
plt.plot(loss_values,'b',label='training loss')
plt.plot(val_loss_values,'r',label='val training loss')
plt.legend()
plt.xlabel("Epochs")

In [ ]:
accuracy_values = history_dict['accuracy']
val_accuracy_values=history_dict['val_accuracy']
plt.plot(val_accuracy_values,'-g',label='val_accuracy')
plt.plot(accuracy_values,'-r',label='acc')
plt.legend()

In [ ]:
print(arg)

In [ ]:
model = keras.Sequential()
stru = arg[0]
reg = arg[1]
tez = arg[2]
f = arg[3]
model.add(keras.layers.Dense(stru[0], input_shape = (16,), activation = f , kernel_regularizer=keras.regularizers.l2(l=reg)))
for i in range(len(stru)-1):
    model.add(keras.layers.Dense(stru[i+1], activation = f , kernel_regularizer=keras.regularizers.l2(l=reg)))
    b = [tez] * stru[i+1]
    w = [b] * stru[i]
    w = np.array(w)
    b = np.zeros(stru[i+1])
    model.layers[i+1].set_weights([w,b])
                

model.add(keras.layers.Dense(26, activation = "softmax"))     
model.compile(optimizer = "adam", loss='categorical_crossentropy', metrics=['accuracy'])
hist = model.fit(x_train, y_train, epochs = 1700, validation_split = 0.15, batch_size = 700, verbose = 0)
loss, acc = model.evaluate(x_test, y_test, verbose = 0)

In [ ]:
history_dict=hist.history

loss_values = history_dict['loss']
val_loss_values=history_dict['val_loss']
plt.figure()
plt.figure()
plt.plot(loss_values,'b',label='training loss')
plt.plot(val_loss_values,'r',label='val training loss')
plt.legend()
plt.xlabel("Epochs")

In [ ]:
accuracy_values = history_dict['accuracy']
val_accuracy_values=history_dict['val_accuracy']
plt.plot(val_accuracy_values,'-g',label='val_accuracy')
plt.plot(accuracy_values,'-r',label='acc')
plt.legend()

In [ ]:
print(acc)

In [ ]:
predict = model.predict(x_train)
conf_y = []
conf_p = []
y_train = np.array(y_train)
for i in range(len(predict)):
    conf_p.append(np.argmax(predict[i], axis=None, out=None))
    conf_y.append(np.argmax(y_train[i], axis=None, out=None))
    
conf_y = np.array(conf_y)
conf_p = np.array(conf_p)

confusion = tf.math.confusion_matrix(conf_y, conf_p)

In [ ]:
confusion = np.array(confusion)
np.set_printoptions(linewidth=np.inf)
print(confusion)

In [ ]:
predict = model.predict(x_test)
conf_y = []
conf_p = []
y_test = np.array(y_test)
for i in range(len(predict)):
    conf_p.append(np.argmax(predict[i], axis=None, out=None))
    conf_y.append(np.argmax(y_train[i], axis=None, out=None))
    
conf_y = np.array(conf_y)
conf_p = np.array(conf_p)

confusion = tf.math.confusion_matrix(conf_y, conf_p)

In [ ]:
confusion = np.array(confusion)
np.set_printoptions(linewidth=np.inf)
print(confusion)